This notebooks takes the model trained in `Colabs/grocery-dataset-model-training.ipynb` notebook and runs inference with it to determine how many products are likely to be present inside a given shelf image. 

## Inital setup

In [1]:
# Which GPU?
!nvidia-smi

Wed Sep 15 05:34:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install TFOD API (TF 1)
%tensorflow_version 1.x
import tensorflow as tf 
print(tf.__version__)

!git clone https://github.com/tensorflow/models.git

% cd models/research
!pip install --upgrade pip
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

TensorFlow 1.x selected.
1.15.2
Cloning into 'models'...
remote: Enumerating objects: 62663, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 62663 (delta 18), reused 53 (delta 14), pack-reused 62604
Receiving objects: 100% (62663/62663), 574.53 MiB | 29.78 MiB/s, done.
Resolving deltas: 100% (43653/43653), done.
/content/models/research
     |████████████████████████████████| 1.6 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at htt

## Gather trained model and test data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r /content/drive/MyDrive/product-detection/fp32/*.pb .
!ls -lh *.pb

-rw------- 1 root root 26M Sep 15 05:37 frozen_inference_graph.pb
-rw------- 1 root root 28M Sep 15 05:37 tflite_graph.pb


In [5]:
!wget -q https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz
!tar xf ShelfImages.tar.gz

In [6]:
!ls -lh ShelfImages/test | head -10

total 101M
-rw-r--r-- 1 1001 1001 1.6M Oct 23  2019 C1_P02_N1_S5_1.JPG
-rw-r--r-- 1 1001 1001 2.3M Oct 23  2019 C1_P02_N2_S2_1.JPG
-rw-r--r-- 1 1001 1001 2.3M Oct 23  2019 C1_P02_N2_S3_1.JPG
-rw-r--r-- 1 1001 1001 1.3M Oct 23  2019 C1_P03_N1_S2_1.JPG
-rw-r--r-- 1 1001 1001 1.6M Oct 23  2019 C1_P03_N1_S3_1.JPG
-rw-r--r-- 1 1001 1001 2.4M Oct 23  2019 C1_P03_N1_S4_1.JPG
-rw-r--r-- 1 1001 1001 1.4M Oct 23  2019 C1_P03_N1_S4_2.JPG
-rw-r--r-- 1 1001 1001 1.1M Oct 23  2019 C1_P03_N2_S2_1.JPG
-rw-r--r-- 1 1001 1001 1.7M Oct 23  2019 C1_P03_N2_S3_1.JPG


## Other imports

In [7]:
from imutils import paths
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import json 

## Image parsing utility

In [8]:
def parse_image(image_path: str) -> np.ndarray:
    """Reads an image and adds a batch dimension."""
    image = plt.imread(image_path).astype(np.uint8)
    image = np.expand_dims(image, 0)
    return image

## Load test image paths

In [9]:
test_image_paths = list(paths.list_images("ShelfImages/test"))
test_image_paths[:5]

['ShelfImages/test/C2_P07_N2_S2_1.JPG',
 'ShelfImages/test/C3_P05_N3_S2_1.JPG',
 'ShelfImages/test/C1_P05_N4_S3_1.JPG',
 'ShelfImages/test/C4_P02_N4_S2_1.JPG',
 'ShelfImages/test/C3_P04_N1_S5_1.JPG']

## Load detection graph

In [10]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile("frozen_inference_graph.pb", "rb") as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Inference utility

In [11]:
def run_inference_for_single_image(image, graph, min_threshold=0.6):
    """Runs detection graph on an image and parses the results."""
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                "num_detections", "detection_boxes", "detection_scores",
                "detection_classes"]:
                tensor_name = key + ":0"
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
                image_tensor = tf.get_default_graph().get_tensor_by_name("image_tensor:0")

            # Run inference
            output_dict = sess.run(tensor_dict,
                                    feed_dict={image_tensor: image})

    # Post-process the results
    output_dict["detection_scores"] = output_dict["detection_scores"][0]
    mask = output_dict["detection_scores"] > min_threshold

    return output_dict["detection_scores"][mask]

## Run bulk inference and prepare JSON file

In [12]:
image_to_products = {}
for image_path in tqdm(test_image_paths):
    image_name = image_path.split("/")[-1]
    image = parse_image(image_path)
    num_products = len(run_inference_for_single_image(image, detection_graph))
    image_to_products[image_name] = num_products

json_string = json.dumps(image_to_products, indent=4) 
with open("image2products.json", "w") as outfile: 
    outfile.write(json_string) 

  0%|          | 0/71 [00:00<?, ?it/s]

In [13]:
pwd

'/content/models/research'

In [ ]:
!head -10 image2products.json

{
    "C2_P03_N2_S3_1.JPG": 22,
    "C1_P10_N1_S5_1.JPG": 51,
    "C3_P01_N2_S3_2.JPG": 17,
    "C4_P03_N1_S3_1.JPG": 33,
    "C2_P04_N3_S2_1.JPG": 21,
    "C3_P03_N2_S4_1.JPG": 49,
    "C3_P04_N1_S5_1.JPG": 40,
    "C4_P08_N2_S2_1.JPG": 24,
    "C4_P03_N1_S4_1.JPG": 45,


## References
* https://github.com/anirbankonar123/CorrosionDetector/blob/master/rust_localization.ipynb